### multi-classification
: multi-nomial classification (다중 분류) : Y값의 범주가 3개 이상인 분류
#### 활성화 함수(Activation function) 으로 softmax함수 가 사용된다

In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
# train data set :
# x_data :  [N,4]  --> [8,4]
x_data = [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
          [4,1,5,5],
          [1,7,5,5],
          [1,2,5,6],
          [1,6,6,6],
          [1,7,7,7]]

# y_data : [N,3] --> [8,3]
y_data = [[0,0,1],  # [2]
          [0,0,1],  # [2]
          [0,0,1],  # [2]
          [0,1,0],  # [1]
          [0,1,0],  # [1]
          [0,1,0],  # [1]
          [1,0,0],  # [0]
          [1,0,0]]  # [0]

x_train = np.array(x_data,dtype=np.float32)
y_train = np.array(y_data,dtype=np.float32)

In [3]:
nb_classes = 3
# 변수 초기화 : weight, bias
# (m,n) * (n,l) = (m,l)   : 행렬의 내적 곱셈 공식
# (8,4) * (4,3) = (8,3)
W = tf.Variable(tf.random.normal([4,nb_classes]),name='weight')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')
print(W)
print(b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.18030666, -0.95028627, -0.03964049],
       [-0.7425406 ,  1.3231523 , -0.61854804],
       [ 0.8540664 , -0.08899953,  2.4488697 ],
       [ 0.762508  ,  1.2659615 ,  0.9801489 ]], dtype=float32)>
<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0.22652863, 0.8106553 , 0.7466094 ], dtype=float32)>


In [4]:
# 예측 함수(hypothesis) : H(X) = softmax(X*W + B)
def logits(X):
    return tf.matmul(X,W) + b

def hypothesis(X):
    return tf.nn.softmax(logits(X))  

In [5]:
#  비용 함수 구현 방법 1: log함수를 사용하여 수식을 직접 표현
# def cost_func():
#     cost = tf.reduce_mean(-tf.reduce_sum(y_train*tf.math.log(hypothesis(x_train)),
#                                          axis=1))
#     return cost

In [6]:
#  비용함수 구현 방법 2 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(x_train),
                                                    labels=y_train)
    cost = tf.reduce_mean(cost_i)
    return cost

In [7]:
# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [8]:
# 학습 시작
print('***** Start Learning!!')
for step in range(5001):
    optimizer.minimize(cost_func,var_list=[W,b])
    if step % 1000 == 0:
        print('%04d'%step,'cost:[',cost_func().numpy(),']',
             ' W:',W.numpy(),' b:',b.numpy())
print('***** Learning Finished!!') 

***** Start Learning!!
0000 cost:[ 5.9294786 ]  W: [[-0.17030679 -0.94028634 -0.04964045]
 [-0.7325406   1.3331522  -0.628548  ]
 [ 0.86406636 -0.07899956  2.4388697 ]
 [ 0.77250797  1.2759615   0.9701489 ]]  b: [0.2365285  0.8206551  0.73660946]
1000 cost:[ 0.314676 ]  W: [[-2.2194262e+00  5.2480602e-01  1.1872499e+00]
 [ 1.2104913e-01  2.2530544e-03 -6.3700818e-02]
 [ 2.2870729e+00  1.1651007e+00  6.7435008e-01]
 [ 1.2760863e+00  1.7768888e+00  3.7673792e-01]]  b: [-2.7187707 -1.5067681  4.3172646]
2000 cost:[ 0.16304696 ]  W: [[-3.7972472   1.3537483   2.2676582 ]
 [ 0.04149133  0.02373761  0.06498493]
 [ 3.5885081   1.049066   -0.5021988 ]
 [ 1.0556881   1.8170282   0.5562775 ]]  b: [-6.1154466 -1.3375355  7.1691594]
3000 cost:[ 0.08965948 ]  W: [[-5.234304    2.1662517   3.2086873 ]
 [-0.04390996  0.06965788  0.1601938 ]
 [ 4.8728323   0.69199103 -1.575506  ]
 [ 0.7372118   2.0572197   0.74264187]]  b: [-9.075829 -1.360418  9.762421]
4000 cost:[ 0.051186908 ]  W: [[-6.5691333   2.

In [10]:
# 예측
# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

# 학습 데이터를 검증 데이터로 동일하게 사용하는 경우
x_test = x_train
y_test = y_train

preds = predict(x_test)
print(preds.numpy())
print(hypothesis(x_test).numpy())
print(tf.argmax(y_test,axis=1).numpy())

[2 2 2 1 1 1 0 0]
[[6.7201379e-15 4.4788053e-06 9.9999547e-01]
 [3.0885360e-11 6.2573748e-03 9.9374264e-01]
 [8.2859493e-18 3.1583805e-02 9.6841621e-01]
 [5.7242958e-16 9.7510344e-01 2.4896551e-02]
 [5.8690526e-02 9.3963611e-01 1.6734067e-03]
 [3.0667935e-02 9.6914309e-01 1.8900653e-04]
 [9.2271829e-01 7.7280879e-02 9.1235523e-07]
 [9.9905401e-01 9.4603549e-04 1.0147042e-10]]
[2 2 2 1 1 1 0 0]
